In [2]:
from os.path import join as pjoin
import os
import numpy as np
from nilearn.image import load_img, new_img_like
from nilearn.masking import apply_mask, unmask
from tqdm import tqdm
import glob
from dimension_encoding.b5k import B5kLoader, compute_noise_ceiling_b5k
import pandas as pd
from scipy.stats import zscore
from dimension_encoding.plotting import plot_bothends_k_rows, plot_topk_rows, plot_list_of_images
from dimension_encoding.glm import LinRegCVPermutation
from dimension_encoding.utils import calc_nc

In [3]:
dl = B5kLoader()
subject = "CSI1"

B5k data loading from: ../data/b5k/


# Inspect embedding

In [4]:
plotdir = os.path.join(os.pardir, 'plots', 'b5k_predicted_dimensions_imagenetcoco')
os.makedirs(plotdir, exist_ok=True)

embedding, fnames = dl.load_predicted_spose_dimensions(image_sets=['imagenet', 'coco'])
stim_dirs = [
    '/SOFTWARE/LABSHARE/bold5000_imagenet', 
    '/SOFTWARE/LABSHARE/bold5000_coco'
    ]
fullpaths = np.array(glob.glob(pjoin(stim_dirs[0], '*')) + glob.glob(pjoin(stim_dirs[1], '*')))
# fullpaths = np.array(glob.glob('/SOFTWARE/LABSHARE/bold5000_coco/*'))

labels = np.loadtxt('../data/66d/labels_short.txt', dtype=str, usecols=0)

In [15]:
# fig = plot_topk_rows(embedding, fullpaths, topk=30)
# fig = plot_bothends_k_rows(embedding, fullpaths)

In [6]:
show_n=100
import matplotlib.pyplot as plt
for dimi, (dim, label) in tqdm(enumerate(zip(embedding.T, labels)), total=len(labels)):
    topims = fullpaths[np.argsort(dim)[-show_n:]]
    fig = plot_list_of_images(topims)
    plt.suptitle(label, y=1.05, fontsize=24)
    fig.show()
    fig.savefig(os.path.join(plotdir, f'dim-{dimi+1}.jpg'), bbox_inches='tight')
    plt.close(fig)

100%|██████████| 66/66 [02:30<00:00,  2.28s/it]


# Test encoding

In [ ]:
X_dims, trial_is, stims = dl.make_dimensions_model(subject)
X_dims = zscore(X_dims, axis=0)

In [8]:
responses = dl.load_responses(subject)
y = responses[trial_is]

In [9]:
# ntrials = y.shape[0]
# for i in range(1,1_000_000):
#     isdivisible = ntrials % i == 0
#     if isdivisible:
#         print(f"ntrials is divisible by {i}")

In [10]:
# just for testing
y, X_dims = y[:21], X_dims[:21]

In [11]:
# # estimate prediction performance
# model = LinRegCVPermutation(nfolds=7, nperm=0)
# r, pval = model.fit_and_permute(X_dims, y)

In [12]:
# Try a fracridge to get cleaner beta maps
# from dimension_encoding.glm import FracRidgeVoxelwise
# from scipy.stats import zscore
# X_dims = np.nan_to_num(zscore(X_dims, axis=0))
# y = np.nan_to_num(zscore(y, axis=0))
# fr = FracRidgeVoxelwise(
#     n_splits=7,
#     test_size=0., # we don't care about prediction performance in this step
#     fracs=np.arange(0.01, 1.01, 0.01),
#     run_pcorr=False,
#     # fracridge_kws={},
# )
# betas, _, r2_test_overfit, best_fracs, _ = fr.tune_and_eval(X_dims, y)